In [21]:
import sys
sys.path.insert(1, '/scratch/cinthiasouza/mv-text-summarizer')

import itertools
import re

import glob, os
import pandas as pd
import json
import spacy
import nltk
import numpy as np
import json
import pickle

import math


from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler


import warnings
warnings.filterwarnings("ignore")

path_base = "/scratch/cinthiasouza/mv-text-summarizer"
path_to_read="/scratch/cinthiasouza/mv-text-summarizer/result/{}/{}_*.csv"

In [22]:
%load_ext autoreload
%autoreload 2

#from src import preprocess
#from src import extract_features
#from src import tokenizer
#from src import create_features_df
from src import transform_data
#from src import loader
#from src import utils
#from src import ensemble_tree_models
#from src import tunning_hyperparametrs as th
#from src import mlp_regressor
#from src import mlp_classifier
from src import summarization
from src import normalization
from src import classifiers
#from src import utils_classification as utils_clf
from src import evaluate_classifiers as ev
#from src import prepare_data
from src import display_results as dr
import joblib
from joblib import Parallel, delayed
#from tensorflow.keras.utils import to_categorical
from src import pipeline_extract_features as pef

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
from tensorflow import keras
from tensorflow.keras.models import model_from_json
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision

In [24]:
def main (
    dataset, tests, num_test, references_df, sections, name_models, columns):
    
    for name_test, approach in tests.items():

        if approach == 'sv':
            index_Xtest='X_test'
        elif approach == 'mv-f1':
            index_Xtest='X_test_f1'
        elif approach == 'mv-f2':
            index_Xtest= 'X_test_f2'
        elif approach == 'embed':
            index_Xtest= 'X_test_embed'

        path_to_save = "/scratch/cinthiasouza/mv-text-summarizer/notebook/{}".format(name_test)

        predictions_proba, models = summarization.load_predict_models(
            dataset, sections, name_models, columns, path_to_save, num_test, index_Xtest=index_Xtest)

        if not os.path.exists("{}/test_{}/summaries".format(path_to_save, num_test)):
                os.makedirs("{}/test_{}/predictions".format(path_to_save, num_test))
                os.makedirs("{}/test_{}/summaries".format(path_to_save, num_test))

        aux = {}
        for section in sections:

            features, scores = summarization.format_data_to_summarize(dataset, section, 'X_test_nf')
            proba_ex1, df_ex1, summaries_ex1, result_ex1 = summarization.pipeline_summarizationv2(
                features, scores, references_df, predictions_proba, section, name_models,
                sort_scores=True, proba=True, ascending=False)

            aux[section] = summaries_ex1

            result_ex1.to_csv("{}/test_{}/summaries/{}.csv".format(path_to_save, num_test, section), index=False)
            proba_ex1.to_csv("{}/test_{}/predictions/{}.csv".format(path_to_save,num_test, section), index=False)
            df_ex1.to_csv("{}/test_{}/predictions/df_{}.csv".format(path_to_save, num_test, section), index=False)

        summaries, result_comb = summarization.combine_summaries_eval(
            aux['introduction'], aux['materials'], aux['conclusion'], references_df)

        result_comb.to_csv("{}/test_{}/summaries/{}.csv".format(path_to_save, num_test, 'comb'), index=False)

In [25]:
path_to_read='/scratch/cinthiasouza/mv-text-summarizer/result_arxiv'
data = 'arxiv'

In [26]:
with open('{}/dataset_{}.pkl'.format(path_to_read,'features'), 'rb') as fp:
    dataset = pickle.load(fp)

In [27]:
sections=['introduction', 'materials', 'conclusion']
name_models = ['knn', 'rf', 'ab', 'gb', 'cb', 'mlp']

In [28]:
references_df = pd.read_csv(f"{path_to_read}/references_df.csv")
references_df['articles'] = references_df['articles'].astype(str)

In [29]:
tests = {f'mv_models_f1-{data}':'mv-f1'}

In [30]:
columns = list(range(0, 383))
columns = list(map(str, columns))

In [31]:
columns = [str(i) for i in range(300)]

In [32]:

from joblib import Parallel, delayed

Parallel(n_jobs=1)(delayed(main)(dataset, tests, num_test, references_df, sections, name_models, columns) for num_test in range(0, 1))
 

Loaded model from disk
Loaded model from disk
Loaded model from disk


[None]

In [14]:
df_intro = pd.read_csv("/scratch/cinthiasouza/mv-text-summarizer/notebook/mv_models_f1-arxi/test_0/summaries/introduction.csv")
df_mat = pd.read_csv("/scratch/cinthiasouza/mv-text-summarizer/notebook/mv_models_f1-plosone/test_0/summaries/materials.csv")
df_conc = pd.read_csv("/scratch/cinthiasouza/mv-text-summarizer/notebook/mv_models_f1-plosone/test_0/summaries/conclusion.csv")
df_comb = pd.read_csv("/scratch/cinthiasouza/mv-text-summarizer/notebook/mv_models_f1-plosone/test_0/summaries/comb.csv")

In [15]:
df_intro.describe()

,articles,knn_rouge-1,knn_rouge-2,knn_rouge-l,rf_rouge-1,rf_rouge-2,rf_rouge-l,ab_rouge-1,ab_rouge-2,ab_rouge-l,gb_rouge-1,gb_rouge-2,gb_rouge-l,cb_rouge-1,cb_rouge-2,cb_rouge-l,mlp_rouge-1,mlp_rouge-2,mlp_rouge-l
count,2.608000e+03,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000
mean,2.511489e+07,0.268858,0.074691,0.206767,0.280385,0.078429,0.210884,0.273377,0.077364,0.209911,0.273128,0.077516,0.210010,0.273964,0.077047,0.209644,0.337259,0.109378,0.238692
std,4.372533e+06,0.076123,0.048382,0.050164,0.077990,0.049678,0.050378,0.076244,0.049287,0.051060,0.076327,0.048892,0.050814,0.076359,0.048557,0.050277,0.078738,0.059372,0.053623
min,1.590631e+07,0.087500,0.000000,0.088233,0.087500,0.000000,0.088233,0.087500,0.000000,0.088233,0.087500,0.000000,0.088233,0.087500,0.000000,0.088233,0.107477,0.000000,0.102638
25%,2.114182e+07,0.213314,0.039584,0.173089,0.223498,0.042300,0.176328,0.218845,0.041902,0.175562,0.218978,0.042395,0.175796,0.219616,0.042153,0.175644,0.283422,0.066460,0.203328
50%,2.483294e+07,0.263616,0.066325,0.201203,0.274379,0.068702,0.205124,0.268539,0.068750,0.204125,0.268446,0.069245,0.204502,0.269172,0.069166,0.205161,0.336554,0.101983,0.231999
75%,2.888977e+07,0.318324,0.098735,0.233314,0.332731,0.103030,0.237034,0.323148,0.101773,0.236034,0.322871,0.101773,0.236080,0.324163,0.101160,0.236190,0.387775,0.142084,0.266523
max,3.484889e+07,0.583333,0.379747,0.529915,0.583691,0.379747,0.499123,0.583691,0.379747,0.499123,0.583691,0.379747,0.499123,0.583691,0.379747,0.499123,0.680672,0.476190,0.567431


In [16]:
df_mat.describe()

,articles,knn_rouge-1,knn_rouge-2,knn_rouge-l,rf_rouge-1,rf_rouge-2,rf_rouge-l,ab_rouge-1,ab_rouge-2,ab_rouge-l,gb_rouge-1,gb_rouge-2,gb_rouge-l,cb_rouge-1,cb_rouge-2,cb_rouge-l,mlp_rouge-1,mlp_rouge-2,mlp_rouge-l
count,2.608000e+03,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000
mean,2.511489e+07,0.219310,0.044739,0.177214,0.198047,0.037615,0.166030,0.190957,0.036096,0.163093,0.194119,0.036758,0.164934,0.182736,0.033643,0.158586,0.242230,0.054136,0.191014
std,4.372533e+06,0.073167,0.038190,0.045833,0.070085,0.033984,0.044844,0.069398,0.033738,0.044347,0.069722,0.033669,0.044195,0.069520,0.032124,0.044672,0.073366,0.041543,0.047063
min,1.590631e+07,0.022346,0.000000,0.035499,0.025000,0.000000,0.035499,0.000000,0.000000,0.000000,0.008368,0.000000,0.020527,0.000000,0.000000,0.000000,0.010204,0.000000,0.023085
25%,2.114182e+07,0.167820,0.017391,0.146830,0.148093,0.013322,0.135371,0.140181,0.012780,0.133507,0.144144,0.012862,0.135252,0.132220,0.011096,0.128664,0.192821,0.023863,0.160311
50%,2.483294e+07,0.215686,0.036123,0.174423,0.193816,0.028891,0.163330,0.184951,0.027260,0.160275,0.187242,0.028537,0.162173,0.178859,0.025478,0.156850,0.240000,0.046647,0.188145
75%,2.888977e+07,0.267153,0.061637,0.202581,0.241888,0.051866,0.190961,0.234558,0.049800,0.188195,0.235379,0.050972,0.189310,0.226887,0.047361,0.185278,0.291080,0.073710,0.217795
max,3.484889e+07,0.522613,0.296296,0.440699,0.518828,0.247934,0.440699,0.477124,0.254980,0.440699,0.475921,0.247934,0.440699,0.461538,0.247934,0.440699,0.512346,0.285714,0.440699


In [17]:
df_conc.describe()

,articles,knn_rouge-1,knn_rouge-2,knn_rouge-l,rf_rouge-1,rf_rouge-2,rf_rouge-l,ab_rouge-1,ab_rouge-2,ab_rouge-l,gb_rouge-1,gb_rouge-2,gb_rouge-l,cb_rouge-1,cb_rouge-2,cb_rouge-l,mlp_rouge-1,mlp_rouge-2,mlp_rouge-l
count,2.608000e+03,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000
mean,2.511489e+07,0.245640,0.062759,0.191229,0.232702,0.057241,0.184981,0.239037,0.059497,0.187869,0.231586,0.056525,0.184442,0.232031,0.056627,0.184229,0.328510,0.107023,0.236396
std,4.372533e+06,0.080348,0.045867,0.050463,0.081219,0.042621,0.050305,0.079692,0.042500,0.048192,0.081443,0.041993,0.049751,0.082060,0.041867,0.049575,0.087724,0.061906,0.057661
min,1.590631e+07,0.000000,0.000000,0.000000,0.006349,0.000000,0.015478,0.012195,0.000000,0.015307,0.006369,0.000000,0.015367,0.012500,0.000000,0.015367,0.000000,0.000000,0.000000
25%,2.114182e+07,0.189864,0.030280,0.158078,0.175297,0.026922,0.153055,0.184840,0.028720,0.156078,0.174189,0.026734,0.151898,0.174109,0.026424,0.151655,0.276074,0.062850,0.201061
50%,2.483294e+07,0.244949,0.052936,0.187493,0.230239,0.049123,0.181548,0.238472,0.050633,0.184575,0.229226,0.047619,0.182217,0.230496,0.048363,0.183000,0.332237,0.097383,0.231372
75%,2.888977e+07,0.297783,0.085106,0.218493,0.285919,0.077135,0.212966,0.292153,0.082323,0.216364,0.284089,0.076765,0.212699,0.287399,0.076923,0.212833,0.384149,0.140231,0.267142
max,3.484889e+07,0.554745,0.355556,0.490711,0.599156,0.365957,0.551429,0.550000,0.331183,0.479929,0.519298,0.336043,0.499355,0.519298,0.336043,0.479929,0.628049,0.515337,0.609050


In [18]:
df_comb.describe()

,articles,knn_rouge-1,knn_rouge-2,knn_rouge-l,rf_rouge-1,rf_rouge-2,rf_rouge-l,ab_rouge-1,ab_rouge-2,ab_rouge-l,gb_rouge-1,gb_rouge-2,gb_rouge-l,cb_rouge-1,cb_rouge-2,cb_rouge-l,mlp_rouge-1,mlp_rouge-2,mlp_rouge-l
count,2.608000e+03,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000
mean,2.511489e+07,0.401390,0.111081,0.246951,0.396385,0.107639,0.243604,0.397358,0.108618,0.245008,0.395489,0.107454,0.244430,0.393771,0.106235,0.243140,0.444257,0.149613,0.272616
std,4.372533e+06,0.060749,0.047964,0.041938,0.060156,0.046401,0.041666,0.060004,0.046881,0.041463,0.059577,0.046273,0.041411,0.060193,0.046355,0.041480,0.071250,0.059391,0.050113
min,1.590631e+07,0.149378,0.005917,0.121494,0.164384,0.007828,0.115474,0.169118,0.013605,0.140089,0.147541,0.011905,0.129032,0.155963,0.011940,0.131702,0.100093,0.006472,0.122520
25%,2.114182e+07,0.364042,0.077220,0.219050,0.357991,0.075244,0.215285,0.359662,0.074890,0.217171,0.357143,0.074074,0.216703,0.355625,0.072990,0.215156,0.401464,0.106513,0.237951
50%,2.483294e+07,0.402985,0.105005,0.241400,0.397820,0.101476,0.238859,0.399159,0.102018,0.239160,0.397196,0.101124,0.239592,0.395137,0.099675,0.238408,0.448676,0.143372,0.268258
75%,2.888977e+07,0.442379,0.137479,0.269815,0.435784,0.134089,0.265580,0.436000,0.135683,0.268182,0.435130,0.133333,0.267209,0.433263,0.132505,0.264757,0.492893,0.187191,0.300678
max,3.484889e+07,0.648421,0.397661,0.522567,0.611111,0.347032,0.510049,0.611670,0.344371,0.491585,0.631373,0.335570,0.492843,0.603687,0.344227,0.490372,0.718254,0.490040,0.554594


In [33]:
df_intro = pd.read_csv("/scratch/cinthiasouza/mv-text-summarizer/notebook/mv_models_f1-arxiv/test_0/summaries/introduction.csv")
df_mat = pd.read_csv("/scratch/cinthiasouza/mv-text-summarizer/notebook/mv_models_f1-arxiv/test_0/summaries/materials.csv")
df_conc = pd.read_csv("/scratch/cinthiasouza/mv-text-summarizer/notebook/mv_models_f1-arxiv/test_0/summaries/conclusion.csv")
df_comb = pd.read_csv("/scratch/cinthiasouza/mv-text-summarizer/notebook/mv_models_f1-arxiv/test_0/summaries/comb.csv")

In [36]:
df_comb.describe()

,knn_rouge-1,knn_rouge-2,knn_rouge-l,rf_rouge-1,rf_rouge-2,rf_rouge-l,ab_rouge-1,ab_rouge-2,ab_rouge-l,gb_rouge-1,gb_rouge-2,gb_rouge-l,cb_rouge-1,cb_rouge-2,cb_rouge-l,mlp_rouge-1,mlp_rouge-2,mlp_rouge-l
count,1112.000000,1112.000000,1112.000000,1112.000000,1112.000000,1112.000000,1112.000000,1112.000000,1112.000000,1112.000000,1112.000000,1112.000000,1112.000000,1112.000000,1112.000000,1112.000000,1112.000000,1112.000000
mean,0.364707,0.115259,0.234726,0.353987,0.114570,0.228390,0.364267,0.117521,0.233730,0.363799,0.118227,0.233873,0.358255,0.116647,0.230785,0.350476,0.122850,0.231212
std,0.084139,0.047905,0.042669,0.089231,0.048021,0.043859,0.087064,0.047117,0.041372,0.087719,0.048350,0.042483,0.087277,0.047009,0.041551,0.092864,0.053388,0.046003
min,0.087102,0.010724,0.072156,0.029323,0.005831,0.026170,0.089010,0.006579,0.071194,0.066496,0.005249,0.072142,0.056437,0.005698,0.072865,0.047923,0.009615,0.071181
25%,0.311354,0.083333,0.208549,0.295473,0.081070,0.202448,0.307513,0.085106,0.208747,0.309041,0.084080,0.207973,0.302194,0.084651,0.205078,0.287680,0.085900,0.203288
50%,0.373744,0.108577,0.233645,0.358632,0.110108,0.227906,0.370057,0.111776,0.233993,0.367826,0.113525,0.233435,0.360805,0.112737,0.229348,0.349352,0.117647,0.230275
75%,0.426218,0.142048,0.259257,0.418343,0.141919,0.255532,0.427594,0.146374,0.258988,0.425975,0.147815,0.259386,0.418659,0.143640,0.257453,0.415684,0.151325,0.257728
max,0.586288,0.410423,0.543989,0.578947,0.343008,0.465128,0.612737,0.381818,0.513474,0.588235,0.319266,0.403357,0.584615,0.330000,0.419349,0.641727,0.394794,0.455157
